In [1]:
import torch
import numpy as np
from torch_geometric.loader import DataLoader

# from Models.GNN import IGCNet
from Utils.training import sr_loss_matrix, average_weights
%reload_ext autoreload
%autoreload 2


# Parameters


In [86]:
num_H = 8
num_test = 20
K = 5 # number of terminals
M = 10 # number of access points
B = 20 # Mhz
D = 1 # km
tau=10
random_matrix = np.random.randn(tau, tau)
U, S, V = np.linalg.svd(random_matrix)

Hb = 15 # Base station height in m
Hm = 1.65 # Mobile height in m
f = 1900 # Frequency in MHz
aL = (1.1 * np.log10(f) - 0.7) * Hm - (1.56 * np.log10(f) - 0.8)
L = 46.3+33.9*np.log10(f)-13.82*np.log10(Hb)-aL

power_f=0.2 # downlink power
rho_p, rho_d = power_f, power_f

# Pd = power_f / 10 ** ((-203.975 + 10 * np.log10(20 * 10 ** 6) + 9) / 10) # normalized receive SNR
Ther_noise = 20000000 * 10**(-17.4) * 10**-3
Pd = 1/Ther_noise
Pu=Pd

d0=0.01 # km
d1=0.05 # km

N = 50

R_cf_min = np.zeros(N)  # Cell Free
R_cf_sum = np.zeros(N)
R_cf_opt_min = np.zeros(N)

In [87]:
num_train = 4
num_test = 2
batchSize = 32
num_rounds = 20

num_epochs = 500
lr = 1e-4
step_size = 5
gamma = 0.9




# Create data loader for training and testing 

In [88]:
from Utils.data_gen import Generate_Input, create_graph

num_AP = M 
Beta_all, Phi_all = Generate_Input(num_train, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
train_data = create_graph(Beta_all, Phi_all, 'het')
train_loader = [
    DataLoader(train_data[i], batch_size=batchSize, shuffle=True)
    for i in range(num_AP)
]

Beta_test, Phi_test = Generate_Input(num_test, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
test_data = create_graph(Beta_test, Phi_test, 'het')
test_loader = [
    DataLoader(test_data[i], batch_size=batchSize, shuffle=False)
    for i in range(num_AP)
]

In [89]:
hidden_channels = 32 # > 4
num_gnn_layers = 2


ap_dim = train_data[0][0]['AP'].x.shape[1]
ue_dim = train_data[0][0]['UE'].x.shape[1]
edge_dim = train_data[0][0]['down'].edge_attr.shape[1]
tt_meta = [('UE', 'up', 'AP'), ('AP', 'down', 'UE')]
dim_dict = {
    'UE': ue_dim,
    'AP': ap_dim,
    'edge': edge_dim,
}

In [90]:
from Models.GNN import APHetNet

model = APHetNet(
    metadata=tt_meta,
    dim_dict=dim_dict,
    out_channels=hidden_channels,
    num_layers=num_gnn_layers,
    hid_layers=hidden_channels
)

In [91]:
from Models.GNN import APHetNet
from Utils.training import train, eval, package_calculate
from Utils.synthetic_graph import return_graph, combine_graph

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

global_model = APHetNet(
    metadata=tt_meta,
    dim_dict=dim_dict,
    out_channels=hidden_channels,
    num_layers=num_gnn_layers,
    hid_layers=hidden_channels
).to(device)
local_models, optimizers = [], []

# Init every client model/optimizer
for each_AP in range(num_AP):
    model = APHetNet(
        metadata=tt_meta,
        dim_dict=dim_dict,
        out_channels=hidden_channels,
        num_layers=num_gnn_layers,
        hid_layers=hidden_channels
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    local_models.append(model)
    optimizers.append(optimizer)

# Main Training

In [ ]:
num_rounds = 20

num_client = num_AP 
num_epochs = 500
eval_round = num_rounds//10

log = []

In [ ]:
import copy
from Utils.training import loss_function, fl_train, fl_eval, get_global_info, distribute_global_info, average_weights


print(f"Starting Federated Learning with {num_client} clients for {num_rounds} rounds")

for round in range(num_rounds):
    # print(f"\n=== Federated Round {round+1}/{num_rounds} ===")
    
    ## 1.Exchange global information
    send_to_server = get_global_info(
        train_loader, local_models, optimizers,
        tau=tau, rho_p=power_f, rho_d=power_f
    )
    response_all = distribute_global_info(send_to_server)
    
    
    ## 2. Training Local models    
    local_weights = []
    total_loss = 0.0
    for model, opt, batches , responses_ap in zip(local_models, optimizers, train_loader, response_all):
        model.train() 
        opt.zero_grad() 
        for epoch in range(num_epochs):
            train_loss = fl_train(
                batches, responses_ap, model, opt,
                tau=tau, rho_p=power_f, rho_d=power_f, num_antenna=N
            )
        local_weights.append(copy.deepcopy(model.state_dict()))
        total_loss += train_loss
    avg_loss = total_loss / num_client
    # print(f"Round {round+1}: Average local training loss = {avg_loss:.6f}")


    ## 3. Update global models
    global_weights = average_weights(glolocal_weights)

    # Broadcast updated global weights to all clients
    for model in local_models:
        model.load_state_dict(global_weights)


    ## 4. Exchange global eval information
    # print("Evaluating global model(s)...")
    send_to_server_eval = get_global_info(
        test_loader, local_models, optimizers,
        tau=tau, rho_p=power_f, rho_d=power_f
    )
    response_all_eval = distribute_global_info(send_to_server_eval)
    total_eval_rate = 0.0
    for client_idx, (model, loader, responses) in enumerate(zip(local_models, test_loader, response_all_eval)):
        model.eval() 
        for epoch in range(num_epochs):
            eval_metrics = fl_eval(
                loader, responses, model,
                tau=tau, rho_p=power_f, rho_d=power_f, num_antenna=N
            )
        total_eval_rate += eval_metrics
    total_eval_rate = total_eval_rate/num_client
        # print(f"Client {client_idx}: {eval_metrics}")
    
    if round%eval_round==0:
        print(f"Round {round+1:02d}/{num_rounds}: Avg Training Rate = {-avg_loss:.6f} | Avg Eval rate = {total_eval_rate:.6f}")
            
    log.append({
        "round": round + 1,
        "train_loss": avg_loss,
        "eval": eval_metrics
    })
    

Starting Federated Learning with 10 clients for 20 rounds


TypeError: fed_avg() missing 1 required positional argument: 'client_weights'

In [210]:
print("\nTraining complete!")
print(f"Final Round | Avg Train Loss: {log[-1]['train_loss']:.6f} | Avg Eval: {log[-1]['eval']}")


Training complete!
Final Round | Avg Train Loss: -2.369643 | Avg Eval: 0.02677050791680813


In [174]:
devcie = batch['UE'].x.device
dtype = batch['UE'].x.dtype

In [176]:
num_antenna = N

device 

num_graphs = batch.num_graphs
num_UEs = batch['UE'].x.shape[0]//num_graphs
num_APs = batch['AP'].x.shape[0]//num_graphs



In [177]:
from Utils.training import component_calculate, variance_calculate

pilot_matrix = batch['UE'].x.reshape(num_graphs, num_UEs, -1)
large_scale = batch['AP','down','UE'].edge_attr.reshape(num_graphs, num_APs, num_UEs)

power = x_dict['UE'].reshape(num_graphs, num_UEs, -1)
power_matrix = power[:,:,-1][:, None, :]

channel_variance = variance_calculate(large_scale, pilot_matrix, tau, rho_p)

DS_k, PC_k, UI_k = component_calculate(power_matrix, channel_variance, large_scale, pilot_matrix, rho=rho_d)

In [178]:
all_DS = [DS_k] + [r['DS'] for r in response]
all_PC = [PC_k] + [r['PC'] for r in response]
all_UI = [UI_k] + [r['UI'] for r in response]

all_DS = torch.cat(all_DS, dim=1)
all_PC = torch.cat(all_PC, dim=1)   # 
all_UI = torch.cat(all_UI, dim=1)   # 

In [193]:
from Utils.training import rate_from_component
rate = rate_from_component(all_DS, all_PC, all_UI, num_antenna)
min,_ = torch.min(rate, dim=1)
print(-torch.mean(min))

tensor(-1.3231, device='cuda:0', grad_fn=<NegBackward0>)


# Testing calculating DS, PC, UI

In [11]:
idx = edge_index['AP', 'down', 'UE']
k = 3
m = 0
s = 3
tmp = s * num_APs * num_UEs + m * num_UEs + k
idx = idx[:, tmp]

In [19]:
batch['AP', 'down', 'UE'].edge_attr[tmp]

tensor([0.0207], device='cuda:0')

In [16]:
largeScale[s, m, k]

tensor(0.0207, device='cuda:0')

In [21]:
num_graphs = batch.num_graphs
num_UEs = x_dict['UE'].shape[0] // num_graphs
num_APs = x_dict['AP'].shape[0] // num_graphs
ue_feature = x_dict['UE'].reshape(num_graphs, num_UEs, -1)
power = ue_feature[:,:, -1][:,None,:]
phiMatrix = ue_feature[:,:, :-1]

largeScale = batch['AP', 'down', 'UE'].edge_attr.reshape(num_graphs, num_APs, num_UEs)

# channelVariance = variance_calculate(largeScale)

In [56]:
# calculating DS PC UI
rho = power_f
num_graphs = 3
num_AP = M
num_UE = K
tau = tau
power = torch.rand(num_graphs, num_AP, num_UE)
channelVariance = torch.rand(num_graphs, num_AP, num_UE)
largeScale = torch.rand(num_graphs, num_AP, num_UE)
phiMatrix = torch.rand(num_graphs, num_UE, tau)

rho_p = power_f

In [82]:
denom = torch.zeros(num_graphs, num_AP, num_UE)
for s in range(num_graphs):
    for k in range(num_UE):
        for m in range(num_AP):
            term1 = 0
            for k_prime in range(num_UE):
                term1 += largeScale[s,m,k_prime] * tmp[s,k_prime, k]
            denom[s,m,k] = tau * rho_p * term1 + 1
tau

10

In [84]:
torch.sum(denom - denom_vec)

tensor(1.2398e-05)

In [91]:
pilotContamination = torch.bmm(
    phiMatrix,
    phiMatrix.transpose(1, 2),
).abs()

DS_all = torch.zeros(num_graphs, num_AP, num_UE)
PC_all = torch.zeros(num_graphs, num_AP, num_UE, num_UE)
UI_all = torch.zeros(num_graphs, num_AP, num_UE, num_UE)

for s in range(num_graphs):
    for m in range(num_AP):
        for k in range(num_UE):
            DS_all[s, m, k] = torch.sqrt(rho * power[s,m,k]) *  channelVariance[s,m,k]
            for k_prime in range(num_UE):
                if k_prime == k: continue
                PC_all[s,m,k_prime,k] = torch.sqrt(rho * power[s,m,k_prime]) * channelVariance[s,m,k_prime]
                PC_all[s,m,k_prime,k] = PC_all[s,m,k_prime,k] * largeScale[s,m,k] / largeScale[s,m,k_prime]
                PC_all[s,m,k_prime,k] = PC_all[s,m,k_prime,k] * pilotContamination[s,k_prime,k]

                UI_all[s,m,k_prime,k] = rho * power[s,m,k_prime] * channelVariance[s,m,k_prime] * largeScale[s,m,k]


In [92]:
from Utils.training import package_calculate
DS_all_matrix, PC_all_matrix, UI_all_matrix = package_calculate(power, channelVariance, largeScale, phiMatrix, power_f)


In [93]:
print(torch.sum(DS_all_matrix - DS_all))
print(torch.sum(PC_all_matrix - PC_all))
print(torch.sum(UI_all_matrix - UI_all))

tensor(0.)
tensor(-1.5063e-06)
tensor(0.)


In [78]:
pilotContamination

tensor([[[6.7123, 3.0449, 3.5054, 2.7120, 5.6616],
         [3.0449, 2.1908, 1.8349, 1.5801, 2.3079],
         [3.5054, 1.8349, 2.2462, 1.4913, 2.9244],
         [2.7120, 1.5801, 1.4913, 1.6840, 2.3059],
         [5.6616, 2.3079, 2.9244, 2.3059, 5.1982]],

        [[4.7962, 3.7576, 2.2543, 4.0618, 2.6975],
         [3.7576, 4.3708, 2.4290, 3.9286, 2.3960],
         [2.2543, 2.4290, 2.1171, 1.7512, 1.6905],
         [4.0618, 3.9286, 1.7512, 5.2153, 2.9763],
         [2.6975, 2.3960, 1.6905, 2.9763, 3.0562]],

        [[3.1328, 2.9414, 2.0053, 2.3504, 1.5182],
         [2.9414, 4.1403, 2.7787, 2.5556, 1.6181],
         [2.0053, 2.7787, 2.7678, 1.9248, 1.1849],
         [2.3504, 2.5556, 1.9248, 2.7166, 1.4954],
         [1.5182, 1.6181, 1.1849, 1.4954, 1.3532]]])

In [ ]:
().shape

torch.Size([3, 10, 5, 5])